# Difference vectors projections onto PCA subspace 
Este código toma una trayectoria completa (previamente fitteada al average) y obtiene los vectores diferencia normalizados. Luego lee los PCA (q también fueron obtenidos en base al average) y proyecta todas los vectores diferencia en la base de los PCA.

In [ ]:
using DataFrames
using Chemfiles
using Gadfly
using Cairo
using Distributions
using Bio

#### Estas celdas son p/ escribir las proyecciones y los vectores diferencia.
#### Si son muchos snapshots, esto toma mucho tiempo. Así q no lo hago.

In [ ]:
# Las layer aceptan solo dataframes
#df_pnum_wt_apo = DataFrame(transpose(pnum_wt_apo));
#writetable("pnum_wt_apo", df_pnum_wt_apo, separator='\t');

#df_pnum_wt_ctv = DataFrame(transpose(pnum_wt_ctv));
#writetable("pnum_wt_ctv", df_pnum_wt_ctv, separator='\t');

#df_pnum_wt_eol = DataFrame(transpose(pnum_wt_eol));
#writetable("pnum_wt_eol", df_pnum_wt_eol, separator='\t');

In [ ]:
#df_projection_wt_apo = DataFrame(projection_wt_apo);
#writetable("projection_wt_apo", df_projection_wt_apo, separator='\t');
#df_projection_wt_apo = 0

#df_projection_wt_ctv = DataFrame(projection_wt_ctv);
#writetable("projection_wt_ctv", df_projection_wt_ctv, separator='\t');
#df_projection_wt_ctv = 0

#df_projection_wt_eol = DataFrame(projection_wt_eol);
#writetable("projection_wt_eol", df_projection_wt_eol, separator='\t');
#df_projection_wt_eol = 0

In [ ]:
# wt_apo
#df_essential_index_wt_apo = DataFrame(modo=essential_index_wt_apo);
#writetable("essential_index_wt_apo", df_essential_index_wt_apo, separator='\t');
#df_essential_index_wt_apo = 0
# wt_ctv
#df_essential_index_wt_ctv = DataFrame(modo=essential_index_wt_ctv);
#writetable("essential_index_wt_ctv", df_essential_index_wt_ctv, separator='\t');
#df_essential_index_wt_ctv = 0
# wt_eol
#df_essential_index_wt_eol = DataFrame(modo=essential_index_wt_eol);
#writetable("essential_index_wt_eol", df_essential_index_wt_eol, separator='\t');
#df_essential_index_wt_eol = 0

In [ ]:
function read_ptraj_modes(file, modes_elements)    
    modes_file=open(file, "r")
    modes_text = readdlm(modes_file, skipstart=0, skipblanks=true, 
    ignore_invalid_chars=true, comments=true, comment_char='\*')
    close(modes_file)

    nmodes = modes_text[1, 5]
    ncoords = convert(Int64, modes_elements)
    lines = ceil(Int64, ncoords/7)
    rest = convert(Int64, ncoords % 7)
    
    eval=Array{Float64}(nmodes);
    mode = Array{Float64}(ncoords, nmodes);
    temp1=Array{Float64}(ncoords, 1);
    temp2 = Array{Float64}(ncoords+(7-rest));

    j=lines + 1 + 2 # 1 p/ q lea la prox linea 2 por el header

    for i=1:nmodes
        eval[i] = modes_text[j, 2]
        temp = transpose(modes_text[(j+1):(lines+j), :])
        temp2 = reshape(temp, ncoords+(7-rest))
        for k=(rest+1):7
            pop!(temp2)
        end
    mode[:, i] = temp2
        j = j + lines + 1
    end
    
    for i=1:nmodes
        mode[: ,i] = mode[:, i] / norm(mode[:, i])
    end
    
    return mode, eval
end

In [ ]:
function read_ptraj_rmsd(file)
    rmsd_file=open(file, "r")
    rmsd_text, rmsd_header = readdlm(rmsd_file, header=true, skipstart=0, skipblanks=true, 
        ignore_invalid_chars=false, comments = false)
    return rmsd_text, rmsd_header
    close(rmsd_file)
end

## wt_apo

In [ ]:
# Preparo variables
wt_apo_tr=Trajectory("/home/german/labo/16/obp/run/wt_apo/avg_fit_wt_apo.nc")
wt_apo_atom_cnt=convert(Int64, natoms(read_step(wt_apo_tr, 1)))
wt_apo_frames_cnt=convert(Int64, nsteps(wt_apo_tr))
wt_apo_avg_tr=Trajectory("/home/german/labo/16/obp/run/wt_apo/avg_wt_apo.pdb");
wt_apo_avg_fr = read(wt_apo_avg_tr);

# Acá leo los indices de los carbonos alfa p/ subsettear los vectores posición.
file_wt_apo_ca_index=open("/home/german/labo/16/obp/run/wt_apo/ca_index_wt_apo");
wt_apo_ca_index_prev = readdlm(file_wt_apo_ca_index);
close(file_wt_apo_ca_index);
wt_apo_ca_index_prev = convert(Array{Int16,2}, wt_apo_ca_index_prev);
wt_apo_ca_index_prev = reshape(wt_apo_ca_index_prev, length(wt_apo_ca_index_prev))

# Numero de aa's y de modos
aa = length(wt_apo_ca_index_prev)
aa3 = 3*length(wt_apo_ca_index_prev)
aa36 = aa3 - 6
# Preparo más variables
diff_vtr_wt_apo = Array{Float32}(aa3, wt_apo_frames_cnt)
modes_wt_apo = Array{Float32}(aa3, aa36)
wt_apo_ca_index = Array{Int16}(aa3)

for i in 1:aa
    j=i*3
    wt_apo_ca_index[j-2] = wt_apo_ca_index_prev[i] * 3 - 2
    wt_apo_ca_index[j-1] = wt_apo_ca_index_prev[i] * 3 - 1
    wt_apo_ca_index[j] = wt_apo_ca_index_prev[i] * 3
end
wt_apo_avg_xyz = getindex(positions(wt_apo_avg_fr), wt_apo_ca_index)

# Obtengo los vectores diferencia de los Calpha entre c/ snapshot y el avg
for i=1:wt_apo_frames_cnt
    wt_apo_fr = read_step(wt_apo_tr, i-1)
    wt_apo_xyz = getindex(positions(wt_apo_fr), wt_apo_ca_index)
    diff_vtr_wt_apo[:, i]= wt_apo_avg_xyz - wt_apo_xyz
    diff_vtr_wt_apo[:, i]= diff_vtr_wt_apo[:, i] / norm(diff_vtr_wt_apo[:, i])
end

# Leo los modos de PCA
modes_wt_apo, evals_wt_apo = read_ptraj_modes("/home/german/labo/16/obp/run/wt_apo/modes_wt_apo", aa3);

# Proyecto los vectores diferencia sobre la base de PCA y obtengo los p-numbers
projection_wt_apo = transpose(modes_wt_apo) * diff_vtr_wt_apo;
projection_wt_apo1 = projection_wt_apo.^4;
pnum_wt_apo = mapslices(sum, projection_wt_apo1, 1);
pnum_wt_apo = map(x -> 1/x, pnum_wt_apo);

# Limpio memoria
wt_apo_tr=0;
modes_wt_apo=0;
diff_vtr_wt_apo=0;
projection_wt_apo1=0;
gc();

## wt_ctv

In [ ]:
# Preparo variables
wt_ctv_tr=Trajectory("/home/german/labo/16/obp/run/wt_ctv/avg_fit_wt_ctv.nc")
wt_ctv_atom_cnt=convert(Int64, natoms(read_step(wt_ctv_tr, 1)))
wt_ctv_frames_cnt=convert(Int64, nsteps(wt_ctv_tr))
wt_ctv_avg_tr=Trajectory("/home/german/labo/16/obp/run/wt_ctv/avg_wt_ctv.pdb");
wt_ctv_avg_fr = read(wt_ctv_avg_tr)

# Acá leo los indices de los carbonos alfa p/ subsettear los vectores posición.
file_wt_ctv_ca_index=open("/home/german/labo/16/obp/run/wt_ctv/ca_index_wt_ctv");
wt_ctv_ca_index_prev = readdlm(file_wt_ctv_ca_index);
close(file_wt_ctv_ca_index);
wt_ctv_ca_index_prev = convert(Array{Int16,2}, wt_ctv_ca_index_prev);
wt_ctv_ca_index_prev = reshape(wt_ctv_ca_index_prev, length(wt_ctv_ca_index_prev))

# Numero de aa's y de modos
aa = length(wt_ctv_ca_index_prev)
aa3 = 3*length(wt_ctv_ca_index_prev)
aa36 = aa3 - 6
# Preparo más variables
diff_vtr_wt_ctv = Array{Float32}(aa3, wt_ctv_frames_cnt)
modes_wt_ctv = Array{Float32}(aa3, aa36)
wt_ctv_ca_index = Array{Int16}(aa3)

for i in 1:aa
    j=i*3
    wt_ctv_ca_index[j-2] = wt_ctv_ca_index_prev[i] * 3 - 2
    wt_ctv_ca_index[j-1] = wt_ctv_ca_index_prev[i] * 3 - 1
    wt_ctv_ca_index[j] = wt_ctv_ca_index_prev[i] * 3
end
wt_ctv_avg_xyz = getindex(positions(wt_ctv_avg_fr), wt_ctv_ca_index)

# Obtengo los vectores diferencia de los Calpha entre c/ snapshot y el avg
for i=1:wt_ctv_frames_cnt
    wt_ctv_fr = read_step(wt_ctv_tr, i-1)
    wt_ctv_xyz = getindex(positions(wt_ctv_fr), wt_ctv_ca_index)
    diff_vtr_wt_ctv[:, i]= wt_ctv_avg_xyz - wt_ctv_xyz
    diff_vtr_wt_ctv[:, i]= diff_vtr_wt_ctv[:, i] / norm(diff_vtr_wt_ctv[:, i])
end

# Leo los modos de PCA
modes_wt_ctv, evals_wt_ctv = read_ptraj_modes("/home/german/labo/16/obp/run/wt_ctv/modes_wt_ctv", aa3);

# Proyecto los vectores diferencia sobre la base de PCA y obtengo los p-numbers
projection_wt_ctv = transpose(modes_wt_ctv) * diff_vtr_wt_ctv;
projection_wt_ctv1 = projection_wt_ctv.^4;
pnum_wt_ctv = mapslices(sum, projection_wt_ctv1, 1);
pnum_wt_ctv = map(x -> 1/x, pnum_wt_ctv);

# Limpio memoria
wt_ctv_tr=0;
modes_wt_ctv=0;
diff_vtr_wt_ctv=0;
projection_wt_ctv1=0;
gc()


## wt_eol

In [ ]:
# Preparo variables
wt_eol_tr=Trajectory("/home/german/labo/16/obp/run/wt_eol/avg_fit_wt_eol.nc")
wt_eol_atom_cnt=convert(Int64, natoms(read_step(wt_eol_tr, 1)))
wt_eol_frames_cnt=convert(Int64, nsteps(wt_eol_tr))
wt_eol_avg_tr=Trajectory("/home/german/labo/16/obp/run/wt_eol/avg_wt_eol.pdb");
wt_eol_avg_fr = read(wt_eol_avg_tr)

# Acá leo los indices de los carbonos alfa p/ subsettear los vectores posición.
file_wt_eol_ca_index=open("/home/german/labo/16/obp/run/wt_eol/ca_index_wt_eol");
wt_eol_ca_index_prev = readdlm(file_wt_eol_ca_index);
close(file_wt_eol_ca_index);
wt_eol_ca_index_prev = convert(Array{Int16,2}, wt_eol_ca_index_prev);
wt_eol_ca_index_prev = reshape(wt_eol_ca_index_prev, length(wt_eol_ca_index_prev))

# Numero de aa's y de modos
aa = length(wt_eol_ca_index_prev)
aa3 = 3*length(wt_eol_ca_index_prev)
aa36 = aa3 - 6
# Preparo más variables
diff_vtr_wt_eol = Array{Float32}(aa3, wt_eol_frames_cnt)
modes_wt_eol = Array{Float32}(aa3, aa36)
wt_eol_ca_index = Array{Int16}(aa3)

for i in 1:aa
    j=i*3
    wt_eol_ca_index[j-2] = wt_eol_ca_index_prev[i] * 3 - 2
    wt_eol_ca_index[j-1] = wt_eol_ca_index_prev[i] * 3 - 1
    wt_eol_ca_index[j] = wt_eol_ca_index_prev[i] * 3
end
wt_eol_avg_xyz = getindex(positions(wt_eol_avg_fr), wt_eol_ca_index)

# Obtengo los vectores diferencia de los Calpha entre c/ snapshot y el avg
for i=1:wt_eol_frames_cnt
    wt_eol_fr = read_step(wt_eol_tr, i-1)
    wt_eol_xyz = getindex(positions(wt_eol_fr), wt_eol_ca_index)
    diff_vtr_wt_eol[:, i]= wt_eol_avg_xyz - wt_eol_xyz
    diff_vtr_wt_eol[:, i]= diff_vtr_wt_eol[:, i] / norm(diff_vtr_wt_eol[:, i])
end

# Leo los modos de PCA
modes_wt_eol, evals_wt_eol = read_ptraj_modes("/home/german/labo/16/obp/run/wt_eol/modes_wt_eol", aa3);

# Proyecto los vectores diferencia sobre la base de PCA y obtengo los p-numbers
projection_wt_eol = transpose(modes_wt_eol) * diff_vtr_wt_eol;
projection_wt_eol1 = projection_wt_eol.^4;
pnum_wt_eol = mapslices(sum, projection_wt_eol1, 1);
pnum_wt_eol = map(x -> 1/x, pnum_wt_eol);

# Limpio memoria
wt_eol_tr=0;
modes_wt_eol=0;
diff_vtr_wt_eol=0;
projection_wt_eol1=0;
gc()

### Plots

In [ ]:
set_default_plot_size(28cm, 16cm)
df_pnum_wt_apo = DataFrame(transpose(pnum_wt_apo));
df_pnum_wt_ctv = DataFrame(transpose(pnum_wt_ctv));
df_pnum_wt_eol = DataFrame(transpose(pnum_wt_eol));

wt_apo_layer = layer(df_pnum_wt_apo, x="x1", Geom.density(), Theme(default_color=colorant"deepskyblue"))
wt_ctv_layer = layer(df_pnum_wt_ctv, x="x1", Geom.density(), Theme(default_color=colorant"blue"))
wt_eol_layer = layer(df_pnum_wt_eol, x="x1", Geom.density(), Theme(default_color=colorant"purple"))

pnum_hist_wt = plot(wt_apo_layer, wt_ctv_layer, wt_eol_layer,
Guide.xlabel("P-number"), Guide.ylabel("Probability density"), Guide.Title("Wt modes P-number"),
Guide.manual_color_key("Legend", ["wt_apo", "wt_ctv", "wt_eol"],
["deepskyblue", "blue", "purple"]),
Coord.Cartesian(xmin=0, xmax=30),
Guide.xticks(ticks = collect(1:2:25), orientation=:horizontal));

pnum_hist_wt

In [ ]:
draw(PDF("pnum_hist_wt.pdf", 20cm, 10cm), pnum_hist_wt);

## Ahora empiezo a ver segun modo

In [ ]:
df_pnum_wt_apo[:x1] = round(df_pnum_wt_apo[:x1]);
df_pnum_wt_ctv[:x1] = round(df_pnum_wt_ctv[:x1]);
df_pnum_wt_eol[:x1] = round(df_pnum_wt_eol[:x1]);
projection_wt_apo1 = abs(projection_wt_apo);
projection_wt_ctv1 = abs(projection_wt_ctv);
projection_wt_eol1 = abs(projection_wt_eol);

### Ordeno las projecciones de los modos 

In [ ]:
# wt_apo
index_wt_apo=Array{Int64}(aa36, wt_apo_frames_cnt-1);
for i=1:wt_apo_frames_cnt-1
    index_wt_apo[:, i] = sortperm(projection_wt_apo1[:, i], rev=true)
end

# wt_ctv
index_wt_ctv=Array{Int64}(aa36, wt_ctv_frames_cnt-1);
for i=1:wt_ctv_frames_cnt-1
    index_wt_ctv[:, i] = sortperm(projection_wt_ctv1[:, i], rev=true)
end

# wt_eol
index_wt_eol=Array{Int64}(aa36, wt_eol_frames_cnt-1);
for i=1:wt_eol_frames_cnt-1
    index_wt_eol[:, i] = sortperm(projection_wt_eol1[:, i], rev=true)
end

### Obtengo los índices de los pnum modos p/ c/ snapshot

In [ ]:
# wt_apo
pmodes_index_wt_apo=Array{Int64}(1);
for i=1:wt_apo_frames_cnt-1
    pnum = convert(Int, df_pnum_wt_apo[i, 1])
    append!(pmodes_index_wt_apo, index_wt_apo[1:pnum, i])
end
shift!(pmodes_index_wt_apo);

# wt_ctv
pmodes_index_wt_ctv=Array{Int64}(1);
for i=1:wt_ctv_frames_cnt-1
    pnum = convert(Int, df_pnum_wt_ctv[i, 1])
    append!(pmodes_index_wt_ctv, index_wt_ctv[1:pnum, i])
end
shift!(pmodes_index_wt_ctv);

# wt_eol
pmodes_index_wt_eol=Array{Int64}(1);
for i=1:wt_eol_frames_cnt-1
    pnum = convert(Int, df_pnum_wt_eol[i, 1])
    append!(pmodes_index_wt_eol, index_wt_eol[1:pnum, i])
end
shift!(pmodes_index_wt_eol);

### Plots

In [ ]:
set_default_plot_size(28cm, 10cm)

# wt_apo
bins_wt_apo, counts_wt_apo = hist(pmodes_index_wt_apo, collect(0:aa36-1))
xmin_wt_apo = bins_wt_apo
xmax_wt_apo = bins_wt_apo .+ 0.33
pop!(xmin_wt_apo)
pop!(xmax_wt_apo)
freq_wt_apo = counts_wt_apo / wt_apo_frames_cnt
pmodes_index_wt_apo_layer = layer(xmin = xmin_wt_apo, xmax = xmax_wt_apo, y = freq_wt_apo, Geom.bar(position=:dodge),
Theme(default_color=colorant"deepskyblue", bar_spacing = 10pt));

# wt_ctv
bins_wt_ctv, counts_wt_ctv = hist(pmodes_index_wt_ctv, collect(0:aa36-1))
xmin_wt_ctv = bins_wt_ctv .+ 0.33
xmax_wt_ctv = bins_wt_ctv .+ 0.66
pop!(xmin_wt_ctv)
pop!(xmax_wt_ctv)
freq_wt_ctv = counts_wt_ctv / wt_ctv_frames_cnt
pmodes_index_wt_ctv_layer = layer(xmin = xmin_wt_ctv, xmax= xmax_wt_ctv, y = freq_wt_ctv, Geom.bar(position=:dodge),
Theme(default_color=colorant"blue", bar_spacing = 10pt))

# wt_eol
bins_wt_eol, counts_wt_eol = hist(pmodes_index_wt_eol, collect(0:aa36-1))
xmin_wt_eol = bins_wt_eol .+ 0.66
xmax_wt_eol = bins_wt_eol .+ 1.0
pop!(xmin_wt_eol)
pop!(xmax_wt_eol)
freq_wt_eol = counts_wt_eol / wt_eol_frames_cnt
pmodes_index_wt_eol_layer = layer(xmin = xmin_wt_eol, xmax= xmax_wt_eol, y = freq_wt_eol, Geom.bar(position=:dodge),
Theme(default_color=colorant"purple", line_width = 1pt, bar_highlight = colorant"purple", bar_spacing = 10pt))

pmodes_hist_wt_apo = plot(pmodes_index_wt_apo_layer,
Guide.xlabel("Mode #"), Guide.ylabel("Probability density"), Guide.Title("Relative probability of each mode"),
Guide.manual_color_key("Legend", ["wt_apo"],
["deepskyblue"]),
Coord.Cartesian(xmin=1, xmax=25),
Guide.xticks(ticks = collect(1:2:25), orientation=:horizontal),
Guide.yticks(ticks = collect(0.0:0.2:1.0), orientation=:horizontal));

pmodes_hist_wt_ctv = plot(pmodes_index_wt_ctv_layer,
Guide.xlabel("Mode #"), Guide.ylabel("Probability density"), Guide.Title("Relative probability of each mode"),
Guide.manual_color_key("Legend", ["wt_ctv"],
["blue"]),
Coord.Cartesian(xmin=1, xmax=25),
Guide.xticks(ticks = collect(1:2:25), orientation=:horizontal),
Guide.yticks(ticks = collect(0.0:0.2:1.0), orientation=:horizontal));

pmodes_hist_wt_eol = plot(pmodes_index_wt_eol_layer,
Guide.xlabel("Mode #"), Guide.ylabel("Probability density"), Guide.Title("Relative probability of each mode"),
Guide.manual_color_key("Legend", ["wt_eol"],
["purple"]),
Coord.Cartesian(xmin=1, xmax=25),
Guide.xticks(ticks = collect(1:2:25), orientation=:horizontal),
Guide.yticks(ticks = collect(0.0:0.2:1.0), orientation=:horizontal));

pmodes_hist_wt_apo
pmodes_hist_wt_ctv
pmodes_hist_wt_eol

In [ ]:
draw(PDF("pmodes_hist_wt.pdf", 20cm, 40cm), vstack(pmodes_hist_wt_apo, pmodes_hist_wt_ctv, pmodes_hist_wt_eol));

# P number de c/ modo según SSE

In [ ]:
# Defino los Calphas de las structs secundarias
α1=collect(1:19)
    l1=collect(20:22)
α2=collect(23:30)
    l2=collect(31:38)
α3=collect(39:51)
    l3=collect(52:62)
α4=collect(63:70)
    l4=collect(71:75)
α5=collect(76:86)
    l5=collect(87:93)
α6=collect(94:106)
    l6=collect(107:110)
α7=collect(111:114)

# Ahora los indices de sus coordenadas xyz
α13 = map(x -> x*3, repeat(α1, inner=[3])) - repeat([2, 1, 0], outer = [length(α1)])
α23 = map(x -> x*3, repeat(α2, inner=[3])) - repeat([2, 1, 0], outer = [length(α2)])
α33 = map(x -> x*3, repeat(α3, inner=[3])) - repeat([2, 1, 0], outer = [length(α3)])
α43 = map(x -> x*3, repeat(α4, inner=[3])) - repeat([2, 1, 0], outer = [length(α4)])
α53 = map(x -> x*3, repeat(α5, inner=[3])) - repeat([2, 1, 0], outer = [length(α5)])
α63 = map(x -> x*3, repeat(α6, inner=[3])) - repeat([2, 1, 0], outer = [length(α6)])
α73 = map(x -> x*3, repeat(α7, inner=[3])) - repeat([2, 1, 0], outer = [length(α7)]);

l13 = map(x -> x*3, repeat(l1, inner=[3])) - repeat([2, 1, 0], outer = [length(l1)])
l23 = map(x -> x*3, repeat(l2, inner=[3])) - repeat([2, 1, 0], outer = [length(l2)])
l33 = map(x -> x*3, repeat(l3, inner=[3])) - repeat([2, 1, 0], outer = [length(l3)])
l43 = map(x -> x*3, repeat(l4, inner=[3])) - repeat([2, 1, 0], outer = [length(l4)])
l53 = map(x -> x*3, repeat(l5, inner=[3])) - repeat([2, 1, 0], outer = [length(l5)])
l63 = map(x -> x*3, repeat(l6, inner=[3])) - repeat([2, 1, 0], outer = [length(l6)]);

sse_count = 13;

## wt_apo

In [ ]:
# Leo los modos de PCA
modes_wt_apo, evals_wt_apo = read_ptraj_modes("/home/german/labo/16/obp/run/wt_apo/modes_wt_apo", aa3);
# Defino subespacio esencial
essential_index_wt_apo = sort(unique(pmodes_index_wt_apo)); # 116 modos
essential_aa36_wt_apo = size(essential_index_wt_apo)[1]
essential_subsp_wt_apo = modes_wt_apo[: , essential_index_wt_apo]

# Subsetteo el essential subspace segun SSE
essential_subsp_α1_wt_apo = essential_subsp_wt_apo[α13, :]
essential_subsp_α2_wt_apo = essential_subsp_wt_apo[α23, :]
essential_subsp_α3_wt_apo = essential_subsp_wt_apo[α33, :]
essential_subsp_α4_wt_apo = essential_subsp_wt_apo[α43, :]
essential_subsp_α5_wt_apo = essential_subsp_wt_apo[α53, :]
essential_subsp_α6_wt_apo = essential_subsp_wt_apo[α63, :]
essential_subsp_α7_wt_apo = essential_subsp_wt_apo[α73, :]

essential_subsp_l1_wt_apo = essential_subsp_wt_apo[l13, :]
essential_subsp_l2_wt_apo = essential_subsp_wt_apo[l23, :]
essential_subsp_l3_wt_apo = essential_subsp_wt_apo[l33, :]
essential_subsp_l4_wt_apo = essential_subsp_wt_apo[l43, :]
essential_subsp_l5_wt_apo = essential_subsp_wt_apo[l53, :]
essential_subsp_l6_wt_apo = essential_subsp_wt_apo[l63, :]

# 1era parte p/ obtener el pnumber de c/ modo
α1_mtx_wt_apo = sum((essential_subsp_α1_wt_apo.^2), 1).^2
α2_mtx_wt_apo = sum((essential_subsp_α2_wt_apo.^2), 1).^2
α3_mtx_wt_apo = sum((essential_subsp_α3_wt_apo.^2), 1).^2
α4_mtx_wt_apo = sum((essential_subsp_α4_wt_apo.^2), 1).^2
α5_mtx_wt_apo = sum((essential_subsp_α5_wt_apo.^2), 1).^2
α6_mtx_wt_apo = sum((essential_subsp_α6_wt_apo.^2), 1).^2
α7_mtx_wt_apo = sum((essential_subsp_α7_wt_apo.^2), 1).^2

l1_mtx_wt_apo = sum((essential_subsp_l1_wt_apo.^2), 1).^2
l2_mtx_wt_apo = sum((essential_subsp_l2_wt_apo.^2), 1).^2
l3_mtx_wt_apo = sum((essential_subsp_l3_wt_apo.^2), 1).^2
l4_mtx_wt_apo = sum((essential_subsp_l4_wt_apo.^2), 1).^2
l5_mtx_wt_apo = sum((essential_subsp_l5_wt_apo.^2), 1).^2
l6_mtx_wt_apo = sum((essential_subsp_l6_wt_apo.^2), 1).^2

# Preparo variables
sse_wt_apo = Array{Float64}[α1_mtx_wt_apo, α2_mtx_wt_apo, α3_mtx_wt_apo, α4_mtx_wt_apo, α5_mtx_wt_apo,
    α6_mtx_wt_apo, α7_mtx_wt_apo, l1_mtx_wt_apo, l2_mtx_wt_apo, l3_mtx_wt_apo,
    l4_mtx_wt_apo, l5_mtx_wt_apo, l6_mtx_wt_apo]
pnum_sse_wt_apo = Array{Float64}(essential_aa36_wt_apo)

# Saco los números de SSE q mueve c/ modo
for i = 1:essential_aa36_wt_apo
    j=1
    temp = 0
    for sse in sse_wt_apo
        temp = temp + sse[i]  
    end
    pnum_sse_wt_apo[i] = 1 / temp
end

## wt_ctv

In [ ]:
# Leo los modos de PCA
modes_wt_ctv, evals_wt_ctv = read_ptraj_modes("/home/german/labo/16/obp/run/wt_ctv/modes_wt_ctv", aa3);
# Defino subespacio esencial
essential_index_wt_ctv = sort(unique(pmodes_index_wt_ctv)); # 116 modos
essential_aa36_wt_ctv = size(essential_index_wt_ctv)[1]
essential_subsp_wt_ctv = modes_wt_ctv[: , essential_index_wt_ctv]

# Subsetteo el essential subspace segun SSE
essential_subsp_α1_wt_ctv = essential_subsp_wt_ctv[α13, :]
essential_subsp_α2_wt_ctv = essential_subsp_wt_ctv[α23, :]
essential_subsp_α3_wt_ctv = essential_subsp_wt_ctv[α33, :]
essential_subsp_α4_wt_ctv = essential_subsp_wt_ctv[α43, :]
essential_subsp_α5_wt_ctv = essential_subsp_wt_ctv[α53, :]
essential_subsp_α6_wt_ctv = essential_subsp_wt_ctv[α63, :]
essential_subsp_α7_wt_ctv = essential_subsp_wt_ctv[α73, :]

essential_subsp_l1_wt_ctv = essential_subsp_wt_ctv[l13, :]
essential_subsp_l2_wt_ctv = essential_subsp_wt_ctv[l23, :]
essential_subsp_l3_wt_ctv = essential_subsp_wt_ctv[l33, :]
essential_subsp_l4_wt_ctv = essential_subsp_wt_ctv[l43, :]
essential_subsp_l5_wt_ctv = essential_subsp_wt_ctv[l53, :]
essential_subsp_l6_wt_ctv = essential_subsp_wt_ctv[l63, :]

# 1era parte p/ obtener el pnumber de c/ modo
α1_mtx_wt_ctv = sum((essential_subsp_α1_wt_ctv.^2), 1).^2
α2_mtx_wt_ctv = sum((essential_subsp_α2_wt_ctv.^2), 1).^2
α3_mtx_wt_ctv = sum((essential_subsp_α3_wt_ctv.^2), 1).^2
α4_mtx_wt_ctv = sum((essential_subsp_α4_wt_ctv.^2), 1).^2
α5_mtx_wt_ctv = sum((essential_subsp_α5_wt_ctv.^2), 1).^2
α6_mtx_wt_ctv = sum((essential_subsp_α6_wt_ctv.^2), 1).^2
α7_mtx_wt_ctv = sum((essential_subsp_α7_wt_ctv.^2), 1).^2

l1_mtx_wt_ctv = sum((essential_subsp_l1_wt_ctv.^2), 1).^2
l2_mtx_wt_ctv = sum((essential_subsp_l2_wt_ctv.^2), 1).^2
l3_mtx_wt_ctv = sum((essential_subsp_l3_wt_ctv.^2), 1).^2
l4_mtx_wt_ctv = sum((essential_subsp_l4_wt_ctv.^2), 1).^2
l5_mtx_wt_ctv = sum((essential_subsp_l5_wt_ctv.^2), 1).^2
l6_mtx_wt_ctv = sum((essential_subsp_l6_wt_ctv.^2), 1).^2

# Preparo variables
sse_wt_ctv = Array{Float64}[α1_mtx_wt_ctv, α2_mtx_wt_ctv, α3_mtx_wt_ctv, α4_mtx_wt_ctv, α5_mtx_wt_ctv,
    α6_mtx_wt_ctv, α7_mtx_wt_ctv, l1_mtx_wt_ctv, l2_mtx_wt_ctv, l3_mtx_wt_ctv,
    l4_mtx_wt_ctv, l5_mtx_wt_ctv, l6_mtx_wt_ctv]
pnum_sse_wt_ctv = Array{Float64}(essential_aa36_wt_ctv)

# Saco los números de SSE q mueve c/ modo
for i = 1:essential_aa36_wt_ctv
    j=1
    temp = 0
    for sse in sse_wt_ctv
        temp = temp + sse[i]  
    end
    pnum_sse_wt_ctv[i] = 1 / temp
end

## wt_eol

In [ ]:
# Leo los modos de PCA
modes_wt_eol, evals_wt_eol = read_ptraj_modes("/home/german/labo/16/obp/run/wt_eol/modes_wt_eol", aa3);
# Defino subespacio esencial
essential_index_wt_eol = sort(unique(pmodes_index_wt_eol)); # 116 modos
essential_aa36_wt_eol = size(essential_index_wt_eol)[1]
essential_subsp_wt_eol = modes_wt_eol[: , essential_index_wt_eol]

# Subsetteo el essential subspace segun SSE
essential_subsp_α1_wt_eol = essential_subsp_wt_eol[α13, :]
essential_subsp_α2_wt_eol = essential_subsp_wt_eol[α23, :]
essential_subsp_α3_wt_eol = essential_subsp_wt_eol[α33, :]
essential_subsp_α4_wt_eol = essential_subsp_wt_eol[α43, :]
essential_subsp_α5_wt_eol = essential_subsp_wt_eol[α53, :]
essential_subsp_α6_wt_eol = essential_subsp_wt_eol[α63, :]
essential_subsp_α7_wt_eol = essential_subsp_wt_eol[α73, :]

essential_subsp_l1_wt_eol = essential_subsp_wt_eol[l13, :]
essential_subsp_l2_wt_eol = essential_subsp_wt_eol[l23, :]
essential_subsp_l3_wt_eol = essential_subsp_wt_eol[l33, :]
essential_subsp_l4_wt_eol = essential_subsp_wt_eol[l43, :]
essential_subsp_l5_wt_eol = essential_subsp_wt_eol[l53, :]
essential_subsp_l6_wt_eol = essential_subsp_wt_eol[l63, :]

# 1era parte p/ obtener el pnumber de c/ modo
α1_mtx_wt_eol = sum((essential_subsp_α1_wt_eol.^2), 1).^2
α2_mtx_wt_eol = sum((essential_subsp_α2_wt_eol.^2), 1).^2
α3_mtx_wt_eol = sum((essential_subsp_α3_wt_eol.^2), 1).^2
α4_mtx_wt_eol = sum((essential_subsp_α4_wt_eol.^2), 1).^2
α5_mtx_wt_eol = sum((essential_subsp_α5_wt_eol.^2), 1).^2
α6_mtx_wt_eol = sum((essential_subsp_α6_wt_eol.^2), 1).^2
α7_mtx_wt_eol = sum((essential_subsp_α7_wt_eol.^2), 1).^2

l1_mtx_wt_eol = sum((essential_subsp_l1_wt_eol.^2), 1).^2
l2_mtx_wt_eol = sum((essential_subsp_l2_wt_eol.^2), 1).^2
l3_mtx_wt_eol = sum((essential_subsp_l3_wt_eol.^2), 1).^2
l4_mtx_wt_eol = sum((essential_subsp_l4_wt_eol.^2), 1).^2
l5_mtx_wt_eol = sum((essential_subsp_l5_wt_eol.^2), 1).^2
l6_mtx_wt_eol = sum((essential_subsp_l6_wt_eol.^2), 1).^2

# Preparo variables
sse_wt_eol = Array{Float64}[α1_mtx_wt_eol, α2_mtx_wt_eol, α3_mtx_wt_eol, α4_mtx_wt_eol, α5_mtx_wt_eol,
    α6_mtx_wt_eol, α7_mtx_wt_eol, l1_mtx_wt_eol, l2_mtx_wt_eol, l3_mtx_wt_eol,
    l4_mtx_wt_eol, l5_mtx_wt_eol, l6_mtx_wt_eol]
pnum_sse_wt_eol = Array{Float64}(essential_aa36_wt_eol)

# Saco los números de SSE q mueve c/ modo
for i = 1:essential_aa36_wt_eol
    j=1
    temp = 0
    for sse in sse_wt_eol
        temp = temp + sse[i]  
    end
    pnum_sse_wt_eol[i] = 1 / temp
end

### Plot!

In [ ]:
# Hago los histogramas
pnum_sse_wt_apo = convert(Array{Int64}, round(pnum_sse_wt_apo))
sse_xmax_wt_apo, sse_freq_wt_apo = hist(pnum_sse_wt_apo, collect(1:1:13))
sse_freq_wt_apo = sse_freq_wt_apo / essential_aa36_wt_apo
sse_xmin_wt_apo = sse_xmax_wt_apo .- .33
pop!(sse_xmin_wt_apo)
pop!(sse_xmax_wt_apo)

pnum_sse_wt_ctv = convert(Array{Int64}, round(pnum_sse_wt_ctv))
sse_xmax_wt_ctv, sse_freq_wt_ctv = hist(pnum_sse_wt_ctv, collect(1:1:13))
sse_freq_wt_ctv = sse_freq_wt_ctv / essential_aa36_wt_ctv
sse_xmin_wt_ctv = sse_xmax_wt_ctv .- .66
pop!(sse_xmin_wt_ctv)
pop!(sse_xmax_wt_ctv)

pnum_sse_wt_eol = convert(Array{Int64}, round(pnum_sse_wt_eol))
sse_xmax_wt_eol, sse_freq_wt_eol = hist(pnum_sse_wt_eol, collect(1:1:13))
sse_freq_wt_eol = sse_freq_wt_eol / essential_aa36_wt_eol
sse_xmin_wt_eol = sse_xmax_wt_eol .- 1
pop!(sse_xmin_wt_eol)
pop!(sse_xmax_wt_eol)

# Los junto en un dataframe
names = [repeat(["wt_apo"], inner=[12]); repeat(["wt_ctv"], inner=[12]); repeat(["wt_eol"], inner=[12])]
datos = [sse_freq_wt_apo; sse_freq_wt_ctv; sse_freq_wt_eol ]
xmins = [sse_xmin_wt_apo; sse_xmin_wt_ctv; sse_xmin_wt_eol]
xmaxs = [sse_xmax_wt_apo; sse_xmax_wt_ctv; sse_xmax_wt_eol]
df_datos = DataFrame(Legend=names, RelativeProbability=datos, xmin=xmins, xmax=xmaxs-.2) # acomodo el xmaxs

pnum_sse_hist = plot(df_datos, x="xmax", y="RelativeProbability", color="Legend", 
Geom.bar(position=:dodge),
Scale.x_continuous,
Theme(bar_spacing=3pt, default_color=colorant"black"),
Guide.Title("P-number in terms of SSEs"),
Coord.Cartesian(xmin=1, xmax=11),
Guide.xticks(ticks = collect(1:1:11), orientation=:horizontal),
Guide.yticks(ticks = collect(0.0:0.1:0.3), orientation=:horizontal), 
Guide.xlabel("SSE participation number"), Guide.ylabel("relative probability"))

In [ ]:
draw(PDF("sse_hist_wt2.pdf", 45cm, 15cm), pnum_sse_hist);

# Ahora hago SSE pero hélices y loops por separado

In [ ]:
# Primero tengo q obtener nuevos indices p/ las hélices y los loops
# hélices
hα13 = collect(1: length(α13))
hα23 = collect(hα13[end]+1: hα13[end]+length(α23))
hα33 = collect(hα23[end]+1: hα23[end]+length(α33))
hα43 = collect(hα33[end]+1: hα33[end]+length(α43))
hα53 = collect(hα43[end]+1: hα43[end]+length(α53))
hα63 = collect(hα53[end]+1: hα53[end]+length(α63))
hα73 = collect(hα63[end]+1: hα63[end]+length(α73));

## helices

## wt_apo

In [ ]:
# Preparo variables
# subsetteo el subespacio esencial q corresponde a las hélices
αnorm_essential_subsp_wt_apo = [essential_subsp_α1_wt_apo; essential_subsp_α2_wt_apo; essential_subsp_α3_wt_apo; 
    essential_subsp_α4_wt_apo; essential_subsp_α5_wt_apo; essential_subsp_α6_wt_apo; essential_subsp_α7_wt_apo];
# lo renormalizo
αnorm_essential_subsp_wt_apo = mapslices(x -> x/norm(x), αnorm_essential_subsp_wt_apo, 1)
# y ahora si, separo los subespacios de c/ hélice
essential_subsp_α1_wt_apo = αnorm_essential_subsp_wt_apo[hα13, :]
essential_subsp_α2_wt_apo = αnorm_essential_subsp_wt_apo[hα23, :]
essential_subsp_α3_wt_apo = αnorm_essential_subsp_wt_apo[hα33, :]
essential_subsp_α4_wt_apo = αnorm_essential_subsp_wt_apo[hα43, :]
essential_subsp_α5_wt_apo = αnorm_essential_subsp_wt_apo[hα53, :]
essential_subsp_α6_wt_apo = αnorm_essential_subsp_wt_apo[hα63, :]
essential_subsp_α7_wt_apo = αnorm_essential_subsp_wt_apo[hα73, :]

# 1era parte p/ obtener el pnumber
α1_mtx_wt_apo = sum((essential_subsp_α1_wt_apo.^2), 1).^2
α2_mtx_wt_apo = sum((essential_subsp_α2_wt_apo.^2), 1).^2
α3_mtx_wt_apo = sum((essential_subsp_α3_wt_apo.^2), 1).^2
α4_mtx_wt_apo = sum((essential_subsp_α4_wt_apo.^2), 1).^2
α5_mtx_wt_apo = sum((essential_subsp_α5_wt_apo.^2), 1).^2
α6_mtx_wt_apo = sum((essential_subsp_α6_wt_apo.^2), 1).^2
α7_mtx_wt_apo = sum((essential_subsp_α7_wt_apo.^2), 1).^2

# junto todo en una lista
sse_wt_apo = Array{Float64}[α1_mtx_wt_apo, α2_mtx_wt_apo, α3_mtx_wt_apo, α4_mtx_wt_apo, α5_mtx_wt_apo,
    α6_mtx_wt_apo, α7_mtx_wt_apo]
# Preparo variable
pnum_sse_wt_apo = Array{Float64}(essential_aa36_wt_apo)

# Saco los números de SSE q mueve c/ modo
for i = 1:essential_aa36_wt_apo
    j=1
    temp = 0
    for sse in sse_wt_apo
        temp = temp + sse[i]  
    end
    pnum_sse_wt_apo[i] = 1 / temp
end

## wt_ctv

In [ ]:
# Preparo variables
# subsetteo el subespacio esencial q corresponde a las hélices
αnorm_essential_subsp_wt_ctv = [essential_subsp_α1_wt_ctv; essential_subsp_α2_wt_ctv; essential_subsp_α3_wt_ctv;
    essential_subsp_α4_wt_ctv; essential_subsp_α5_wt_ctv; essential_subsp_α6_wt_ctv; essential_subsp_α7_wt_ctv];
# lo renormalizo
αnorm_essential_subsp_wt_ctv = mapslices(x -> x/norm(x), αnorm_essential_subsp_wt_ctv, 1)
# y ahora si, separo los subespacios de c/ hélice
essential_subsp_α1_wt_ctv = αnorm_essential_subsp_wt_ctv[hα13, :]
essential_subsp_α2_wt_ctv = αnorm_essential_subsp_wt_ctv[hα23, :]
essential_subsp_α3_wt_ctv = αnorm_essential_subsp_wt_ctv[hα33, :]
essential_subsp_α4_wt_ctv = αnorm_essential_subsp_wt_ctv[hα43, :]
essential_subsp_α5_wt_ctv = αnorm_essential_subsp_wt_ctv[hα53, :]
essential_subsp_α6_wt_ctv = αnorm_essential_subsp_wt_ctv[hα63, :]
essential_subsp_α7_wt_ctv = αnorm_essential_subsp_wt_ctv[hα73, :]

# 1era parte p/ obtener el pnumber
α1_mtx_wt_ctv = sum((essential_subsp_α1_wt_ctv.^2), 1).^2
α2_mtx_wt_ctv = sum((essential_subsp_α2_wt_ctv.^2), 1).^2
α3_mtx_wt_ctv = sum((essential_subsp_α3_wt_ctv.^2), 1).^2
α4_mtx_wt_ctv = sum((essential_subsp_α4_wt_ctv.^2), 1).^2
α5_mtx_wt_ctv = sum((essential_subsp_α5_wt_ctv.^2), 1).^2
α6_mtx_wt_ctv = sum((essential_subsp_α6_wt_ctv.^2), 1).^2
α7_mtx_wt_ctv = sum((essential_subsp_α7_wt_ctv.^2), 1).^2

# junto todo en una lista
sse_wt_ctv = Array{Float64}[α1_mtx_wt_ctv, α2_mtx_wt_ctv, α3_mtx_wt_ctv, α4_mtx_wt_ctv, α5_mtx_wt_ctv,
    α6_mtx_wt_ctv, α7_mtx_wt_ctv]
pnum_sse_wt_ctv = Array{Float64}(essential_aa36_wt_ctv)

# Saco los números de SSE q mueve c/ modo
for i = 1:essential_aa36_wt_ctv
    j=1
    temp = 0
    for sse in sse_wt_ctv
        temp = temp + sse[i]  
    end
    pnum_sse_wt_ctv[i] = 1 / temp
end



## wt_eol

In [ ]:
# Preparo variables
# subsetteo el subespacio esencial q corresponde a las hélices
αnorm_essential_subsp_wt_eol = [essential_subsp_α1_wt_eol; essential_subsp_α2_wt_eol; essential_subsp_α3_wt_eol;
    essential_subsp_α4_wt_eol; essential_subsp_α5_wt_eol; essential_subsp_α6_wt_eol; essential_subsp_α7_wt_eol];
# lo renormalizo
αnorm_essential_subsp_wt_eol = mapslices(x -> x/norm(x), αnorm_essential_subsp_wt_eol, 1)
# y ahora si, separo los subespacios de c/ hélice
essential_subsp_α1_wt_eol = αnorm_essential_subsp_wt_eol[hα13, :]
essential_subsp_α2_wt_eol = αnorm_essential_subsp_wt_eol[hα23, :]
essential_subsp_α3_wt_eol = αnorm_essential_subsp_wt_eol[hα33, :]
essential_subsp_α4_wt_eol = αnorm_essential_subsp_wt_eol[hα43, :]
essential_subsp_α5_wt_eol = αnorm_essential_subsp_wt_eol[hα53, :]
essential_subsp_α6_wt_eol = αnorm_essential_subsp_wt_eol[hα63, :]
essential_subsp_α7_wt_eol = αnorm_essential_subsp_wt_eol[hα73, :]

# 1era parte p/ obtener el pnumber
α1_mtx_wt_eol = sum((essential_subsp_α1_wt_eol.^2), 1).^2
α2_mtx_wt_eol = sum((essential_subsp_α2_wt_eol.^2), 1).^2
α3_mtx_wt_eol = sum((essential_subsp_α3_wt_eol.^2), 1).^2
α4_mtx_wt_eol = sum((essential_subsp_α4_wt_eol.^2), 1).^2
α5_mtx_wt_eol = sum((essential_subsp_α5_wt_eol.^2), 1).^2
α6_mtx_wt_eol = sum((essential_subsp_α6_wt_eol.^2), 1).^2
α7_mtx_wt_eol = sum((essential_subsp_α7_wt_eol.^2), 1).^2

# junto todo en una lista
sse_wt_eol = Array{Float64}[α1_mtx_wt_eol, α2_mtx_wt_eol, α3_mtx_wt_eol, α4_mtx_wt_eol, α5_mtx_wt_eol,
    α6_mtx_wt_eol, α7_mtx_wt_eol]
# Preparo variable
pnum_sse_wt_eol = Array{Float64}(essential_aa36_wt_eol)

# Saco los números de SSE q mueve c/ modo
for i = 1:essential_aa36_wt_eol
    j=1
    temp = 0
    for sse in sse_wt_eol
        temp = temp + sse[i]  
    end
    pnum_sse_wt_eol[i] = 1 / temp
end

## Plot de helices

In [ ]:
# otro plotteo

# Hago los histogramas
pnum_sse_wt_apo = convert(Array{Int64}, round(pnum_sse_wt_apo))
sse_xmax_wt_apo, sse_freq_wt_apo = hist(pnum_sse_wt_apo, collect(1:1:8))
sse_freq_wt_apo = sse_freq_wt_apo / essential_aa36_wt_apo
sse_xmin_wt_apo = sse_xmax_wt_apo .- .33
pop!(sse_xmin_wt_apo)
pop!(sse_xmax_wt_apo)

pnum_sse_wt_ctv = convert(Array{Int64}, round(pnum_sse_wt_ctv))
sse_xmax_wt_ctv, sse_freq_wt_ctv = hist(pnum_sse_wt_ctv, collect(1:1:8))
sse_freq_wt_ctv = sse_freq_wt_ctv / essential_aa36_wt_ctv
sse_xmin_wt_ctv = sse_xmax_wt_ctv .- .66
pop!(sse_xmin_wt_ctv)
pop!(sse_xmax_wt_ctv)

pnum_sse_wt_eol = convert(Array{Int64}, round(pnum_sse_wt_eol))
sse_xmax_wt_eol, sse_freq_wt_eol = hist(pnum_sse_wt_eol, collect(1:1:8))
sse_freq_wt_eol = sse_freq_wt_eol / essential_aa36_wt_eol
sse_xmin_wt_eol = sse_xmax_wt_eol .- 1
pop!(sse_xmin_wt_eol)
pop!(sse_xmax_wt_eol)


# Los junto en un dataframe
names = [repeat(["wt_apo"], inner=[7]); repeat(["wt_ctv"], inner=[7]); repeat(["wt_eol"], inner=[7])]
datos = [sse_freq_wt_apo; sse_freq_wt_ctv; sse_freq_wt_eol ]
xmins = [sse_xmin_wt_apo; sse_xmin_wt_ctv; sse_xmin_wt_eol]
xmaxs = [sse_xmax_wt_apo; sse_xmax_wt_ctv; sse_xmax_wt_eol]
df_datos_helices = DataFrame(Legend=names, RelativeProbability=datos, xmin=xmins, xmax=xmaxs+.5) # acomodo el xmaxs

pnum_helices_hist = plot(df_datos_helices, x="xmax", y="RelativeProbability", color="Legend", 
Geom.bar(position=:dodge),
Scale.x_continuous,
Theme(bar_spacing=3pt, default_color=colorant"black"),
Guide.Title("P-number in terms of helices"),
Coord.Cartesian(xmin=0, xmax=8),
Guide.xticks(ticks = collect(1:1:8), orientation=:horizontal),
Guide.yticks(ticks = collect(0.0:0.1:0.4), orientation=:horizontal), 
Guide.xlabel("helices participation number"), Guide.ylabel("relative probability"))

##  loops

In [ ]:
# loops
ll13 = collect(1: length(l13))
ll23 = collect(ll13[end]+1: ll13[end]+length(l23))
ll33 = collect(ll23[end]+1: ll23[end]+length(l33))
ll43 = collect(ll33[end]+1: ll33[end]+length(l43))
ll53 = collect(ll43[end]+1: ll43[end]+length(l53))
ll63 = collect(ll53[end]+1: ll53[end]+length(l63));

## wt_apo

In [ ]:
# Preparo variables
# subsetteo el subespacio esencial q corresponde a las hélices
lnorm_essential_subsp_wt_apo = [essential_subsp_l1_wt_apo; essential_subsp_l2_wt_apo; essential_subsp_l3_wt_apo;
    essential_subsp_l4_wt_apo; essential_subsp_l5_wt_apo; essential_subsp_l6_wt_apo];
# lo renormalizo
lnorm_essential_subsp_wt_apo = mapslices(x -> x/norm(x), lnorm_essential_subsp_wt_apo, 1)
# y ahora si, separo los subespacios de c/ hélice
essential_subsp_l1_wt_apo = lnorm_essential_subsp_wt_apo[ll13, :]
essential_subsp_l2_wt_apo = lnorm_essential_subsp_wt_apo[ll23, :]
essential_subsp_l3_wt_apo = lnorm_essential_subsp_wt_apo[ll33, :]
essential_subsp_l4_wt_apo = lnorm_essential_subsp_wt_apo[ll43, :]
essential_subsp_l5_wt_apo = lnorm_essential_subsp_wt_apo[ll53, :]
essential_subsp_l6_wt_apo = lnorm_essential_subsp_wt_apo[ll63, :]

# 1era parte p/ obtener el pnumber
l1_mtx_wt_apo = sum((essential_subsp_l1_wt_apo.^2), 1).^2
l2_mtx_wt_apo = sum((essential_subsp_l2_wt_apo.^2), 1).^2
l3_mtx_wt_apo = sum((essential_subsp_l3_wt_apo.^2), 1).^2
l4_mtx_wt_apo = sum((essential_subsp_l4_wt_apo.^2), 1).^2
l5_mtx_wt_apo = sum((essential_subsp_l5_wt_apo.^2), 1).^2
l6_mtx_wt_apo = sum((essential_subsp_l6_wt_apo.^2), 1).^2

# junto todo en una lista
sse_wt_apo = Array{Float64}[l1_mtx_wt_apo, l2_mtx_wt_apo, l3_mtx_wt_apo, l4_mtx_wt_apo, l5_mtx_wt_apo,
    l6_mtx_wt_apo]
# Preparo variable
pnum_sse_wt_apo = Array{Float64}(essential_aa36_wt_apo)

# Saco los números de SSE q mueve c/ modo
for i = 1:essential_aa36_wt_apo
    j=1
    temp = 0
    for sse in sse_wt_apo
        temp = temp + sse[i]
    end
    pnum_sse_wt_apo[i] = 1 / temp
end

## wt_ctv

In [ ]:
# Preparo variables
# subsetteo el subespacio esencial q corresponde a las hélices
lnorm_essential_subsp_wt_ctv = [essential_subsp_l1_wt_ctv; essential_subsp_l2_wt_ctv; essential_subsp_l3_wt_ctv;
    essential_subsp_l4_wt_ctv; essential_subsp_l5_wt_ctv; essential_subsp_l6_wt_ctv];
# lo renormalizo
lnorm_essential_subsp_wt_ctv = mapslices(x -> x/norm(x), lnorm_essential_subsp_wt_ctv, 1)
# y ahora si, separo los subespacios de c/ hélice
essential_subsp_l1_wt_ctv = lnorm_essential_subsp_wt_ctv[ll13, :]
essential_subsp_l2_wt_ctv = lnorm_essential_subsp_wt_ctv[ll23, :]
essential_subsp_l3_wt_ctv = lnorm_essential_subsp_wt_ctv[ll33, :]
essential_subsp_l4_wt_ctv = lnorm_essential_subsp_wt_ctv[ll43, :]
essential_subsp_l5_wt_ctv = lnorm_essential_subsp_wt_ctv[ll53, :]
essential_subsp_l6_wt_ctv = lnorm_essential_subsp_wt_ctv[ll63, :]

# 1era parte p/ obtener el pnumber
l1_mtx_wt_ctv = sum((essential_subsp_l1_wt_ctv.^2), 1).^2
l2_mtx_wt_ctv = sum((essential_subsp_l2_wt_ctv.^2), 1).^2
l3_mtx_wt_ctv = sum((essential_subsp_l3_wt_ctv.^2), 1).^2
l4_mtx_wt_ctv = sum((essential_subsp_l4_wt_ctv.^2), 1).^2
l5_mtx_wt_ctv = sum((essential_subsp_l5_wt_ctv.^2), 1).^2
l6_mtx_wt_ctv = sum((essential_subsp_l6_wt_ctv.^2), 1).^2

# junto todo en una lista
sse_wt_ctv = Array{Float64}[l1_mtx_wt_ctv, l2_mtx_wt_ctv, l3_mtx_wt_ctv, l4_mtx_wt_ctv, l5_mtx_wt_ctv,
    l6_mtx_wt_ctv]
# Preparo variable
pnum_sse_wt_ctv = Array{Float64}(essential_aa36_wt_ctv)

# Saco los números de SSE q mueve c/ modo
for i = 1:essential_aa36_wt_ctv
    j=1
    temp = 0
    for sse in sse_wt_ctv
        temp = temp + sse[i]
    end
    pnum_sse_wt_ctv[i] = 1 / temp
end

## wt_eol

In [ ]:
# Preparo variables
# subsetteo el subespacio esencial q corresponde a las hélices
lnorm_essential_subsp_wt_eol = [essential_subsp_l1_wt_eol; essential_subsp_l2_wt_eol; essential_subsp_l3_wt_eol;
    essential_subsp_l4_wt_eol; essential_subsp_l5_wt_eol; essential_subsp_l6_wt_eol];
# lo renormalizo
lnorm_essential_subsp_wt_eol = mapslices(x -> x/norm(x), lnorm_essential_subsp_wt_eol, 1)
# y ahora si, separo los subespacios de c/ hélice
essential_subsp_l1_wt_eol = lnorm_essential_subsp_wt_eol[ll13, :]
essential_subsp_l2_wt_eol = lnorm_essential_subsp_wt_eol[ll23, :]
essential_subsp_l3_wt_eol = lnorm_essential_subsp_wt_eol[ll33, :]
essential_subsp_l4_wt_eol = lnorm_essential_subsp_wt_eol[ll43, :]
essential_subsp_l5_wt_eol = lnorm_essential_subsp_wt_eol[ll53, :]
essential_subsp_l6_wt_eol = lnorm_essential_subsp_wt_eol[ll63, :]

# 1era parte p/ obtener el pnumber
l1_mtx_wt_eol = sum((essential_subsp_l1_wt_eol.^2), 1).^2
l2_mtx_wt_eol = sum((essential_subsp_l2_wt_eol.^2), 1).^2
l3_mtx_wt_eol = sum((essential_subsp_l3_wt_eol.^2), 1).^2
l4_mtx_wt_eol = sum((essential_subsp_l4_wt_eol.^2), 1).^2
l5_mtx_wt_eol = sum((essential_subsp_l5_wt_eol.^2), 1).^2
l6_mtx_wt_eol = sum((essential_subsp_l6_wt_eol.^2), 1).^2

# junto todo en una lista
sse_wt_eol = Array{Float64}[l1_mtx_wt_eol, l2_mtx_wt_eol, l3_mtx_wt_eol, l4_mtx_wt_eol, l5_mtx_wt_eol,
    l6_mtx_wt_eol]
# Preparo variable
pnum_sse_wt_eol = Array{Float64}(essential_aa36_wt_eol)

# Saco los números de SSE q mueve c/ modo
for i = 1:essential_aa36_wt_eol
    j=1
    temp = 0
    for sse in sse_wt_eol
        temp = temp + sse[i]
    end
    pnum_sse_wt_eol[i] = 1 / temp
end


## Plot de loops

In [ ]:
# otro plotteo

# Hago los histogramas
pnum_sse_wt_apo = convert(Array{Int64}, round(pnum_sse_wt_apo))
sse_xmax_wt_apo, sse_freq_wt_apo = hist(pnum_sse_wt_apo, collect(1:1:7))
sse_freq_wt_apo = sse_freq_wt_apo / essential_aa36_wt_apo
sse_xmin_wt_apo = sse_xmax_wt_apo .- .33
pop!(sse_xmin_wt_apo)
pop!(sse_xmax_wt_apo)

pnum_sse_wt_ctv = convert(Array{Int64}, round(pnum_sse_wt_ctv))
sse_xmax_wt_ctv, sse_freq_wt_ctv = hist(pnum_sse_wt_ctv, collect(1:1:7))
sse_freq_wt_ctv = sse_freq_wt_ctv / essential_aa36_wt_ctv
sse_xmin_wt_ctv = sse_xmax_wt_ctv .- .66
pop!(sse_xmin_wt_ctv)
pop!(sse_xmax_wt_ctv)

pnum_sse_wt_eol = convert(Array{Int64}, round(pnum_sse_wt_eol))
sse_xmax_wt_eol, sse_freq_wt_eol = hist(pnum_sse_wt_eol, collect(1:1:7))
sse_freq_wt_eol = sse_freq_wt_eol / essential_aa36_wt_eol
sse_xmin_wt_eol = sse_xmax_wt_eol .- 1
pop!(sse_xmin_wt_eol)
pop!(sse_xmax_wt_eol)


# Los junto en un dataframe
names = [repeat(["wt_apo"], inner=[6]); repeat(["wt_ctv"], inner=[6]); repeat(["wt_eol"], inner=[6])]
datos = [sse_freq_wt_apo; sse_freq_wt_ctv; sse_freq_wt_eol ]
xmins = [sse_xmin_wt_apo; sse_xmin_wt_ctv; sse_xmin_wt_eol]
xmaxs = [sse_xmax_wt_apo; sse_xmax_wt_ctv; sse_xmax_wt_eol]
df_datos_loops = DataFrame(Legend=names, RelativeProbability=datos, xmin=xmins, xmax=xmaxs+.5) # acomodo el xmaxs

#pnum_loops_hist = plot(df_datos_loops, xmin="xmin", xmax="xmax", y="RelativeProbability", color="Legend",
pnum_loops_hist = plot(df_datos_loops, x="xmax", y="RelativeProbability", color="Legend",
Geom.bar(position=:dodge),
Scale.x_continuous,
Theme(bar_spacing=3pt, default_color=colorant"black"),
Guide.Title("P-number in terms of loops"),
Coord.Cartesian(xmin=0, xmax=7),
Guide.xticks(ticks = collect(1:1:6), orientation=:horizontal),
Guide.yticks(ticks = collect(0.0:0.1:0.5), orientation=:horizontal), 
Guide.xlabel("loops participation number"), Guide.ylabel("relative probability"))

In [ ]:
draw(PDF("loops-vs-helices_hist_wt2.pdf", 45cm, 15cm), hstack(pnum_helices_hist, pnum_loops_hist));

# Tabla de comparación

## Avg contributions de c/ SSE al subespacio

In [ ]:
# Subsetteo el essential subspace segun SSE
essential_subsp_α1_wt_apo = essential_subsp_wt_apo[α13, :]
essential_subsp_α2_wt_apo = essential_subsp_wt_apo[α23, :]
essential_subsp_α3_wt_apo = essential_subsp_wt_apo[α33, :]
essential_subsp_α4_wt_apo = essential_subsp_wt_apo[α43, :]
essential_subsp_α5_wt_apo = essential_subsp_wt_apo[α53, :]
essential_subsp_α6_wt_apo = essential_subsp_wt_apo[α63, :]
essential_subsp_α7_wt_apo = essential_subsp_wt_apo[α73, :]

essential_subsp_l1_wt_apo = essential_subsp_wt_apo[l13, :]
essential_subsp_l2_wt_apo = essential_subsp_wt_apo[l23, :]
essential_subsp_l3_wt_apo = essential_subsp_wt_apo[l33, :]
essential_subsp_l4_wt_apo = essential_subsp_wt_apo[l43, :]
essential_subsp_l5_wt_apo = essential_subsp_wt_apo[l53, :]
essential_subsp_l6_wt_apo = essential_subsp_wt_apo[l63, :]

# compilo todo en una mtx
contribution_wt_apo = [ sum((essential_subsp_α1_wt_apo.^2), 1).^2;
α2_mtx_wt_apo = sum((essential_subsp_α2_wt_apo.^2), 1).^2; α3_mtx_wt_apo = sum((essential_subsp_α3_wt_apo.^2), 1).^2;
α4_mtx_wt_apo = sum((essential_subsp_α4_wt_apo.^2), 1).^2; α5_mtx_wt_apo = sum((essential_subsp_α5_wt_apo.^2), 1).^2;
α6_mtx_wt_apo = sum((essential_subsp_α6_wt_apo.^2), 1).^2; α7_mtx_wt_apo = sum((essential_subsp_α7_wt_apo.^2), 1).^2;
l1_mtx_wt_apo = sum((essential_subsp_l1_wt_apo.^2), 1).^2; l2_mtx_wt_apo = sum((essential_subsp_l2_wt_apo.^2), 1).^2;
l3_mtx_wt_apo = sum((essential_subsp_l3_wt_apo.^2), 1).^2; l4_mtx_wt_apo = sum((essential_subsp_l4_wt_apo.^2), 1).^2;
l5_mtx_wt_apo = sum((essential_subsp_l5_wt_apo.^2), 1).^2; l6_mtx_wt_apo = sum((essential_subsp_l6_wt_apo.^2), 1).^2 ];

# c/ fila corresponde a un SSE. Las normalizo según la longitud del SSE
contribution_wt_apo = diagm([ 1/length(α1), 1/length(α2), 1/length(α3), 1/length(α4), 1/length(α5), 1/length(α6), 1/length(α7),
    1/length(l1), 1/length(l2), 1/length(l3), 1/length(l4), 1/length(l5), 1/length(l6) ]) * contribution_wt_apo;

# hago promedio ponderado por la incidencia de c/ modo
s_wt_apo = contribution_wt_apo * freq_wt_apo[essential_index_wt_apo];
# determino la contribución de c/ SSE
s_wt_apo = s_wt_apo ./ sum(s_wt_apo) .* 100;

In [ ]:
# Subsetteo el essential subspace segun SSE
essential_subsp_α1_wt_ctv = essential_subsp_wt_ctv[α13, :]
essential_subsp_α2_wt_ctv = essential_subsp_wt_ctv[α23, :]
essential_subsp_α3_wt_ctv = essential_subsp_wt_ctv[α33, :]
essential_subsp_α4_wt_ctv = essential_subsp_wt_ctv[α43, :]
essential_subsp_α5_wt_ctv = essential_subsp_wt_ctv[α53, :]
essential_subsp_α6_wt_ctv = essential_subsp_wt_ctv[α63, :]
essential_subsp_α7_wt_ctv = essential_subsp_wt_ctv[α73, :]

essential_subsp_l1_wt_ctv = essential_subsp_wt_ctv[l13, :]
essential_subsp_l2_wt_ctv = essential_subsp_wt_ctv[l23, :]
essential_subsp_l3_wt_ctv = essential_subsp_wt_ctv[l33, :]
essential_subsp_l4_wt_ctv = essential_subsp_wt_ctv[l43, :]
essential_subsp_l5_wt_ctv = essential_subsp_wt_ctv[l53, :]
essential_subsp_l6_wt_ctv = essential_subsp_wt_ctv[l63, :]

# compilo todo en una mtx
contribution_wt_ctv = [ sum((essential_subsp_α1_wt_ctv.^2), 1).^2;
α2_mtx_wt_ctv = sum((essential_subsp_α2_wt_ctv.^2), 1).^2; α3_mtx_wt_ctv = sum((essential_subsp_α3_wt_ctv.^2), 1).^2;
α4_mtx_wt_ctv = sum((essential_subsp_α4_wt_ctv.^2), 1).^2; α5_mtx_wt_ctv = sum((essential_subsp_α5_wt_ctv.^2), 1).^2;
α6_mtx_wt_ctv = sum((essential_subsp_α6_wt_ctv.^2), 1).^2; α7_mtx_wt_ctv = sum((essential_subsp_α7_wt_ctv.^2), 1).^2;
l1_mtx_wt_ctv = sum((essential_subsp_l1_wt_ctv.^2), 1).^2; l2_mtx_wt_ctv = sum((essential_subsp_l2_wt_ctv.^2), 1).^2;
l3_mtx_wt_ctv = sum((essential_subsp_l3_wt_ctv.^2), 1).^2; l4_mtx_wt_ctv = sum((essential_subsp_l4_wt_ctv.^2), 1).^2;
l5_mtx_wt_ctv = sum((essential_subsp_l5_wt_ctv.^2), 1).^2; l6_mtx_wt_ctv = sum((essential_subsp_l6_wt_ctv.^2), 1).^2 ];

# c/ fila corresponde a un SSE. Las normalizo según la longitud del SSE
contribution_wt_ctv = diagm([ 1/length(α1), 1/length(α2), 1/length(α3), 1/length(α4), 1/length(α5), 1/length(α6), 1/length(α7),
    1/length(l1), 1/length(l2), 1/length(l3), 1/length(l4), 1/length(l5), 1/length(l6) ]) * contribution_wt_ctv

# hago promedio ponderado por la incidencia de c/ modo
s_wt_ctv = contribution_wt_ctv * freq_wt_ctv[essential_index_wt_ctv]
# determino la contribución de c/ SSE
s_wt_ctv = s_wt_ctv ./ sum(s_wt_ctv) .* 100;

In [ ]:
# Subsetteo el essential subspace segun SSE
essential_subsp_α1_wt_eol = essential_subsp_wt_eol[α13, :]
essential_subsp_α2_wt_eol = essential_subsp_wt_eol[α23, :]
essential_subsp_α3_wt_eol = essential_subsp_wt_eol[α33, :]
essential_subsp_α4_wt_eol = essential_subsp_wt_eol[α43, :]
essential_subsp_α5_wt_eol = essential_subsp_wt_eol[α53, :]
essential_subsp_α6_wt_eol = essential_subsp_wt_eol[α63, :]
essential_subsp_α7_wt_eol = essential_subsp_wt_eol[α73, :]

essential_subsp_l1_wt_eol = essential_subsp_wt_eol[l13, :]
essential_subsp_l2_wt_eol = essential_subsp_wt_eol[l23, :]
essential_subsp_l3_wt_eol = essential_subsp_wt_eol[l33, :]
essential_subsp_l4_wt_eol = essential_subsp_wt_eol[l43, :]
essential_subsp_l5_wt_eol = essential_subsp_wt_eol[l53, :]
essential_subsp_l6_wt_eol = essential_subsp_wt_eol[l63, :]

# compilo todo en una mtx
contribution_wt_eol = [ sum((essential_subsp_α1_wt_eol.^2), 1).^2;
α2_mtx_wt_eol = sum((essential_subsp_α2_wt_eol.^2), 1).^2; α3_mtx_wt_eol = sum((essential_subsp_α3_wt_eol.^2), 1).^2;
α4_mtx_wt_eol = sum((essential_subsp_α4_wt_eol.^2), 1).^2; α5_mtx_wt_eol = sum((essential_subsp_α5_wt_eol.^2), 1).^2;
α6_mtx_wt_eol = sum((essential_subsp_α6_wt_eol.^2), 1).^2; α7_mtx_wt_eol = sum((essential_subsp_α7_wt_eol.^2), 1).^2;
l1_mtx_wt_eol = sum((essential_subsp_l1_wt_eol.^2), 1).^2; l2_mtx_wt_eol = sum((essential_subsp_l2_wt_eol.^2), 1).^2;
l3_mtx_wt_eol = sum((essential_subsp_l3_wt_eol.^2), 1).^2; l4_mtx_wt_eol = sum((essential_subsp_l4_wt_eol.^2), 1).^2;
l5_mtx_wt_eol = sum((essential_subsp_l5_wt_eol.^2), 1).^2; l6_mtx_wt_eol = sum((essential_subsp_l6_wt_eol.^2), 1).^2 ];

# c/ fila corresponde a un SSE. Las normalizo según la longitud del SSE
contribution_wt_eol = diagm([ 1/length(α1), 1/length(α2), 1/length(α3), 1/length(α4), 1/length(α5), 1/length(α6), 1/length(α7),
    1/length(l1), 1/length(l2), 1/length(l3), 1/length(l4), 1/length(l5), 1/length(l6) ]) * contribution_wt_eol

# hago promedio ponderado por la incidencia de c/ modo
s_wt_eol = contribution_wt_eol * freq_wt_eol[essential_index_wt_eol]
# determino la contribución de c/ SSE
s_wt_eol = s_wt_eol ./ sum(s_wt_eol) .* 100;

## RMSD de SSEs

## wt_apo

In [ ]:
# Leo los rmsds
rmsd_a1_wt_apo, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_apo/rmsd/a1_rmsd_wt_apo")
        rmsd_l1_wt_apo, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_apo/rmsd/l1_rmsd_wt_apo")
rmsd_a2_wt_apo, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_apo/rmsd/a2_rmsd_wt_apo")
        rmsd_l2_wt_apo, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_apo/rmsd/a2_rmsd_wt_apo")
rmsd_a3_wt_apo, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_apo/rmsd/a3_rmsd_wt_apo")
        rmsd_l3_wt_apo, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_apo/rmsd/a3_rmsd_wt_apo")
rmsd_a4_wt_apo, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_apo/rmsd/a4_rmsd_wt_apo")
        rmsd_l4_wt_apo, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_apo/rmsd/a4_rmsd_wt_apo")
rmsd_a5_wt_apo, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_apo/rmsd/a5_rmsd_wt_apo")
        rmsd_l5_wt_apo, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_apo/rmsd/a5_rmsd_wt_apo")
rmsd_a6_wt_apo, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_apo/rmsd/a6_rmsd_wt_apo")
        rmsd_l6_wt_apo, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_apo/rmsd/a6_rmsd_wt_apo")
rmsd_a7_wt_apo, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_apo/rmsd/a7_rmsd_wt_apo");

# Los compilo en un array
rmsd_wt_apo = [mean(rmsd_a1_wt_apo[:, 2]); mean(rmsd_a2_wt_apo[:, 2]); mean(rmsd_a3_wt_apo[:, 2]); mean(rmsd_a4_wt_apo[:, 2]);
mean(rmsd_a5_wt_apo[:, 2]); mean(rmsd_a6_wt_apo[:, 2]); mean(rmsd_a7_wt_apo[:, 2]); mean(rmsd_l1_wt_apo[:, 2]);
mean(rmsd_l2_wt_apo[:, 2]); mean(rmsd_l3_wt_apo[:, 2]); mean(rmsd_l4_wt_apo[:, 2]); mean(rmsd_l5_wt_apo[:, 2]);
mean(rmsd_l6_wt_apo[:, 2])];

## wt_ctv

In [ ]:
# Leo los rmsds
rmsd_a1_wt_ctv, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_ctv/rmsd/a1_rmsd_wt_ctv")
        rmsd_l1_wt_ctv, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_ctv/rmsd/l1_rmsd_wt_ctv")
rmsd_a2_wt_ctv, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_ctv/rmsd/a2_rmsd_wt_ctv")
        rmsd_l2_wt_ctv, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_ctv/rmsd/a2_rmsd_wt_ctv")
rmsd_a3_wt_ctv, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_ctv/rmsd/a3_rmsd_wt_ctv")
        rmsd_l3_wt_ctv, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_ctv/rmsd/a3_rmsd_wt_ctv")
rmsd_a4_wt_ctv, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_ctv/rmsd/a4_rmsd_wt_ctv")
        rmsd_l4_wt_ctv, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_ctv/rmsd/a4_rmsd_wt_ctv")
rmsd_a5_wt_ctv, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_ctv/rmsd/a5_rmsd_wt_ctv")
        rmsd_l5_wt_ctv, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_ctv/rmsd/a5_rmsd_wt_ctv")
rmsd_a6_wt_ctv, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_ctv/rmsd/a6_rmsd_wt_ctv")
        rmsd_l6_wt_ctv, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_ctv/rmsd/a6_rmsd_wt_ctv")
rmsd_a7_wt_ctv, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_ctv/rmsd/a7_rmsd_wt_ctv");

# Los compilo en un array
rmsd_wt_ctv = [mean(rmsd_a1_wt_ctv[:, 2]); mean(rmsd_a2_wt_ctv[:, 2]); mean(rmsd_a3_wt_ctv[:, 2]); mean(rmsd_a4_wt_ctv[:, 2]);
mean(rmsd_a5_wt_ctv[:, 2]); mean(rmsd_a6_wt_ctv[:, 2]); mean(rmsd_a7_wt_ctv[:, 2]); mean(rmsd_l1_wt_ctv[:, 2]);
mean(rmsd_l2_wt_ctv[:, 2]); mean(rmsd_l3_wt_ctv[:, 2]); mean(rmsd_l4_wt_ctv[:, 2]); mean(rmsd_l5_wt_ctv[:, 2]);
mean(rmsd_l6_wt_ctv[:, 2])];

## wt_eol

In [ ]:
# Leo los rmsds
rmsd_a1_wt_eol, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_eol/rmsd/a1_rmsd_wt_eol")
        rmsd_l1_wt_eol, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_eol/rmsd/l1_rmsd_wt_eol")
rmsd_a2_wt_eol, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_eol/rmsd/a2_rmsd_wt_eol")
        rmsd_l2_wt_eol, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_eol/rmsd/a2_rmsd_wt_eol")
rmsd_a3_wt_eol, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_eol/rmsd/a3_rmsd_wt_eol")
        rmsd_l3_wt_eol, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_eol/rmsd/a3_rmsd_wt_eol")
rmsd_a4_wt_eol, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_eol/rmsd/a4_rmsd_wt_eol")
        rmsd_l4_wt_eol, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_eol/rmsd/a4_rmsd_wt_eol")
rmsd_a5_wt_eol, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_eol/rmsd/a5_rmsd_wt_eol")
        rmsd_l5_wt_eol, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_eol/rmsd/a5_rmsd_wt_eol")
rmsd_a6_wt_eol, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_eol/rmsd/a6_rmsd_wt_eol")
        rmsd_l6_wt_eol, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_eol/rmsd/a6_rmsd_wt_eol")
rmsd_a7_wt_eol, header = read_ptraj_rmsd("/home/german/labo/16/obp/run/wt_eol/rmsd/a7_rmsd_wt_eol");

# Los compilo en un array
rmsd_wt_eol = [mean(rmsd_a1_wt_eol[:, 2]); mean(rmsd_a2_wt_eol[:, 2]); mean(rmsd_a3_wt_eol[:, 2]); mean(rmsd_a4_wt_eol[:, 2]);
mean(rmsd_a5_wt_eol[:, 2]); mean(rmsd_a6_wt_eol[:, 2]); mean(rmsd_a7_wt_eol[:, 2]); mean(rmsd_l1_wt_eol[:, 2]);
mean(rmsd_l2_wt_eol[:, 2]); mean(rmsd_l3_wt_eol[:, 2]); mean(rmsd_l4_wt_eol[:, 2]); mean(rmsd_l5_wt_eol[:, 2]);
mean(rmsd_l6_wt_eol[:, 2])];

In [ ]:
# Ahora hago la tabla con los resultados

# hago la columna de residuos
resi = [ string(α1[1], "-", α1[end]), string(α2[1], "-", α2[end]), string(α3[1], "-", α3[end]),
string(α4[1], "-", α4[end]), string(α5[1], "-", α5[end]), string(α6[1], "-", α6[end]),
string(α7[1], "-", α7[end]), string(l1[1], "-", l1[end]), string(l2[1], "-", l2[end]),
string(l3[1], "-", l3[end]), string(l4[1], "-", l4[end]), string(l5[1], "-", l5[end]), string(l6[1], "-", l6[end]) ]

# junto todo en un data frame
df_tabla = DataFrame(Index = collect(1:sse_count), Residues = resi,
SSE = ["α1", "α2", "α3", "α4", "α5", "α6", "α7", "L1", "L2", "L3", "L4", "L5", "L6"],
SWtApo = round(s_wt_apo, 2), SWtCtv = round(s_wt_ctv, 2), SWtEol = round(s_wt_eol, 2), RMSDWtApo= round(rmsd_wt_apo, 2),
RMSDWtCtv = round(rmsd_wt_ctv, 2), RMSDWtEol = round(rmsd_wt_eol, 2))

# escribo el data frame en un file
writetable("tabla_1", df_tabla, separator='\t')

df_tabla